In [1]:
import h5py
import os
import nibabel as nib
import torch
import numpy as np

datapath = "/home/lm/Schreibtisch/Uni/MedicalML/pytorch-3dunet/data/training"

files = os.listdir(datapath)
files = filter(lambda x: x.endswith("_orig.nii.gz"), files)
files = list(map(lambda x: x.replace("_orig.nii.gz", ""), files))
files

['A094_R',
 'A047',
 'A130_L',
 'A097',
 'A118',
 'A003',
 'A073',
 'A028',
 'A081',
 'A021',
 'A046',
 'A115',
 'A049',
 'A077',
 'A013',
 'A074',
 'A070',
 'A105_L',
 'A067',
 'A012',
 'A127',
 'A060',
 'A035',
 'A009',
 'A087',
 'A095',
 'A057',
 'A044',
 'A024',
 'A084',
 'A041',
 'A059_L',
 'A085',
 'A076',
 'A054',
 'A121',
 'A033',
 'A014',
 'PA5',
 'A025',
 'A001',
 'A092',
 'A038_R',
 'A038_L',
 'A062_L',
 'A015',
 'A138',
 'A135',
 'A124',
 'A079',
 'A089_R',
 'A051_R',
 'A010',
 'A056',
 'A113',
 'A098',
 'A103',
 'A029',
 'A078_L',
 'A083',
 'A043',
 'A119',
 'A108',
 'A134',
 'A038_M',
 'A086',
 'A082',
 'A072',
 'A050',
 'A091_R',
 'A023_R',
 'A068',
 'A120',
 'A071',
 'A008',
 'A093',
 'A040',
 'A114',
 'A096_R',
 'A031',
 'A006',
 'A096_L',
 'A100',
 'A129',
 'A130_R',
 'PA6',
 'A016',
 'A133',
 'A042',
 'A018',
 'A026',
 'A017_L',
 'A032',
 'A066',
 'A123',
 'A064',
 'A112',
 'A088',
 'A078_R',
 'A126',
 'A019',
 'A137',
 'A136',
 'A005',
 'A027',
 'A045',
 'A105_R',
 

In [2]:
images = []
labels = []
affines = []
cases = []

for case in files:
        image_raw = nib.load(f"{datapath}/{case}_orig.nii.gz")
        mask_raw = nib.load(f"{datapath}/{case}_masks.nii.gz")

        image = image_raw.get_fdata()
        mask = mask_raw.get_fdata()

        if image.shape != mask.shape:
                print("Dont match")
                continue

        if image.shape[2] < 220:

                print("Before", image.shape, mask.shape)
                diff = 220 - image.shape[2]
                print("Diff", diff, case)
                
                image = np.concatenate([image] + [image[:, :, -1][:, :, np.newaxis]] * diff, axis=2)
                mask = np.concatenate([mask] + [mask[:, :, -1][:, :, np.newaxis]] * diff, axis=2)

                print("after", image.shape, mask.shape)
                
        if image.shape[2] > 220:
                print("Before", image.shape, mask.shape)
                image = image[:, :, :220]
                mask = mask[:, :, :220]
                print("after", image.shape, mask.shape)

        affine = image_raw.affine

        images.append(image)
        labels.append(mask)
        affines.append(affine)
        cases.append(case)

Before (256, 256, 228) (256, 256, 228)
after (256, 256, 220) (256, 256, 220)
Before (256, 256, 221) (256, 256, 221)
after (256, 256, 220) (256, 256, 220)
Before (256, 256, 228) (256, 256, 228)
after (256, 256, 220) (256, 256, 220)
Before (256, 256, 218) (256, 256, 218)
Diff 2 A114
after (256, 256, 220) (256, 256, 220)
Before (256, 256, 228) (256, 256, 228)
after (256, 256, 220) (256, 256, 220)


In [3]:
hf = h5py.File("data.h5", "w")

hf.create_dataset("images", data=images)
hf.create_dataset("masks", data=labels)
hf.create_dataset("affines", data=affines)
hf.create_dataset("names", data=cases)

hf.close()



: 

: 